# Perkins V - Calculating Performance Indicators by CIP

In [1]:
# Author: Matthew Fikes
# Modified: 3/11/22
import pandas as pd


### This code will calculate the Perkins 2P1 and 3P1 performance indicators.

#### If your file is not named CTEA.xlsx the code will prompt you to enter the filename. This assumes values in CTEA file are numeric and not descriptions (e.g. non-completion status codes are 4 and 6). This was designed only for the CTEA-1 but can be modified.

In [2]:
file = 'UTICA_AB.xlsx'

try:
    data = pd.read_excel(file)
    print('File {0} loaded successfully'.format(file))
except:
    file = input('CTEA.xlsx not found. Please enter the full filename: ')
    try:
        data = pd.read_excel(file)
        print('File {0} loaded successfully'.format(file))
    except:
        print('No file found')
        exit
    

File UTICA_AB.xlsx loaded successfully


## Static Variables
These can be modified to suit your individual file. Targets for 1P1, 2P1, and 3P1 ar also included so they can be easily changed.

In [3]:
target_1 = .4975
target_2 = .2075
target_3 = .1675

In [4]:
# fields to use for calculations from merged CTEA 1A/B - can be renamed if your fields are different
cip_field = 'CTEA 1A / CIP Code / DYN||8774_DYN'
status_field = 'CTEA 1A / YRENDSTAT_ID / DYN||8765_DYN'
credit_field = 'CTEA 1A / Credits Earned / DYN||8763_DYN'
emp_field = 'CTEA 1B / EMPSTAT_ID / DYN||8785_DYN'
educ_field = 'CTEA 1B / EDUCSTAT_ID / DYN||8784_DYN'
id_field = 'CTEA 1A / Student ID / DYN||8761_DYN'
gender_field = 'CTEA 1A / Gender / DYN||8770_DYN'

In [5]:
# special population fields
disab_field = 'CTEA 1A / Disabled / DYN||8766_DYN'
displ_field = 'CTEA 1A / DISPLACED_HM / DYN||8776_DYN'
econdis_field = 'CTEA 1A / Economic Disadvantage / DYN||8777_DYN'
homels_field = 'CTEA 1A / Homeless / DYN||8769_DYN'
lim_eng_field = 'CTEA 1A / Limited English / DYN||8780_DYN'
migr_field = 'CTEA 1A / Migrant / DYN||8772_DYN'
sparnt_field = 'CTEA 1A / Single Parent / DYN||8775_DYN'
youth_ao_field = 'CTEA 1A / Youth Aged Out / DYN||8779_DYN'
youth_af_field = 'CTEA 1A / Youth Armed Forces / DYN||8762_DYN'
spop_fields = [disab_field,displ_field,econdis_field,homels_field,lim_eng_field,migr_field,sparnt_field,youth_ao_field,youth_af_field]

### Get Non-traditional Crosswalk

In [6]:
xwalk_url = 'https://s3.amazonaws.com/PCRN/docs/REVISED_FINAL-2020-Nontraditional-Crosswalk-6-9-2021.xlsx'
try:
    print('Downloading Non-traditional crosswalk from ',xwalk_url)
    nontrad_xwalk = pd.read_excel(xwalk_url)
    print('Loaded successfully')
except:
    print('Unable to connect to ',xwalk_url)

Loaded successfully


### Join CTEA to crosswalk and separate completers

In [7]:
# merge crosswalk with loaded CTEA file, joined by CIP
new_data = nontrad_xwalk[['CIP 6 2020','Female','Male']].merge(data,left_on='CIP 6 2020',right_on=cip_field,how='right')

In [8]:
# splits completers based on status codes 4 and 6 and total credits >=12
completer_set = new_data[(new_data[status_field].isin([4,6])) | (new_data[credit_field]>=12)] #make numerator data

In [27]:
# uncomment to make dataframe of noncompleters
#non_completers = pd.merge(new_data,completer_set,how='outer',on=id_field,indicator=True)
#noncomp_df = non_completers.loc[non_completers._merge == 'left_only']

# 1P1 Performance Indicator

In [28]:
num_1p1 = completer_set[(completer_set[status_field].isin([4,6])) & 
                        ((completer_set[emp_field].isin([1,2,3,7,8,9]))|
                          (completer_set[educ_field]==1))]
den_1p1 = completer_set[(completer_set[status_field].isin([4,6]))]

In [116]:
# Calculate performance for each special population by CIP, return values below target
rows_1p1 = []
for i in spop_fields:

    lim_num = pd.DataFrame(num_1p1[(num_1p1[i]==1)])
    lim_den = pd.DataFrame(den_1p1[(den_1p1[i]==1)])

    num = pd.DataFrame(lim_num).filter([cip_field,i]).groupby(cip_field).count()
    dem = pd.DataFrame(lim_den).filter([cip_field,i]).groupby(cip_field).count()
    pct = num.divide(dem)
    underperf = pct[(pct.values<=target_1)]
    
    rows_1p1.append(underperf.reset_index())
    
spop_1p1 = pd.concat(rows_1p1).dropna(axis=1,how='all').groupby('CTEA 1A / CIP Code / DYN||8774_DYN').max().reset_index()

In [137]:
num_grp_1 = num_1p1.groupby(cip_field)[id_field].count()
den_grp_1 = den_1p1.groupby(cip_field)[id_field].count()
percents_1p1 = num_grp_1.divide(den_grp_1,fill_value=0)

## MOST RECENT PERFORMANCE DATA BY PROGRAM

In [250]:
pf1p1a = zip(percents_1p1.index,percents_1p1.values)
for i in pf1p1a:
    print('{0:.4f}---{1:.2f}%'.format(i[0],i[1]*100))

11.0201---33.33%
11.0401---80.00%
11.0701---66.67%
11.1003---75.00%
15.0201---88.89%
15.0303---66.67%
15.0407---0.00%
15.0801---100.00%
15.0805---33.33%
15.1302---50.00%
31.0101---62.50%
43.0103---36.67%
43.0107---31.25%
43.0114---50.00%
43.0204---70.00%
44.0000---70.59%
45.0702---100.00%
47.0101---55.56%
47.0201---0.00%
48.0508---36.96%
48.0510---50.00%
50.0102---23.08%
50.0406---50.00%
50.0409---60.00%
50.0410---21.05%
51.0000---33.33%
51.0707---55.56%
51.0907---62.50%
51.0908---16.67%
51.1501---72.22%
51.3801---43.75%
52.0201---60.55%
52.0205---69.23%
52.0302---45.45%
52.0401---40.00%
52.0801---80.00%


## PROGRAMS NOT MEETING TARGET

In [33]:
p_unmet_1p1 = pd.DataFrame(percents_1p1[(percents_1p1.values<=target_1)]).reset_index().set_index('CTEA 1A / CIP Code / DYN||8774_DYN')
p_unmet_1p1_pf = percents_1p1[(percents_1p1.values<=target_1)]

In [253]:
pf1p1b = zip(p_unmet_1p1_pf.index,p_unmet_1p1_pf.values)
for i in pf1p1b:
    print('{0:.4f}---{1:.2f}%'.format(i[0],i[1]*100))

11.0201---33.33%
15.0407---0.00%
15.0805---33.33%
43.0103---36.67%
43.0107---31.25%
47.0201---0.00%
48.0508---36.96%
50.0102---23.08%
50.0410---21.05%
51.0000---33.33%
51.0908---16.67%
51.3801---43.75%
52.0302---45.45%
52.0401---40.00%


## SPECIAL POPULATIONS NOT MEETING TARGET WITHIN PROGRAMS

In [198]:
# Special Pops falling below 1P1 target, by CIP
unmet_spop_1p1 = spop_1p1.set_index('CTEA 1A / CIP Code / DYN||8774_DYN')


In [199]:
for items in rows_1p1:
    try:
        print(items.columns[1].split(' / ')[1])
        for v in items.values:
            cip = v[0]
            meas = v[1]
            print('{0:.4f}---{1:.2f}%'.format(cip,meas*100))

    except:
        print('NA')
    

Disabled
50.0102---40.00%
50.0410---33.33%
52.0201---33.33%
DISPLACED_HM
51.3801---33.33%
Economic Disadvantage
15.0805---25.00%
43.0103---37.50%
43.0107---35.71%
48.0508---33.33%
50.0102---20.00%
50.0410---20.00%
51.3801---48.65%
52.0401---28.57%
Homeless
Limited English
52.0302---25.00%
Migrant
Single Parent
51.3801---25.00%
52.0201---42.86%
Youth Aged Out
Youth Armed Forces


# 2P1 Performance Indicator

In [36]:
num_2p1 = completer_set[(completer_set[status_field].isin([4,6]))]
den_2p1 = completer_set[(completer_set[status_field].isin([4,5,6]))]
num_grp_2 = num_2p1.groupby(cip_field)[id_field].count()
den_grp_2 = den_2p1.groupby(cip_field)[id_field].count()

In [38]:
# Calculate performance for each special population by CIP, return values below target
rows_2p1 = []
for i in spop_fields:

    lim_num = pd.DataFrame(num_2p1[(num_2p1[i]==1)])
    lim_den = pd.DataFrame(den_2p1[(den_2p1[i]==1)])

    num = pd.DataFrame(lim_num).filter([cip_field,i]).groupby(cip_field).count()
    dem = pd.DataFrame(lim_den).filter([cip_field,i]).groupby(cip_field).count()
    pct = num.divide(dem)
    underperf = pct[(pct.values<=target_2)]
    
    rows_2p1.append(underperf.reset_index())
    
spop_2p1 = pd.concat(rows_2p1).dropna(axis=1,how='all').groupby('CTEA 1A / CIP Code / DYN||8774_DYN').max().reset_index()

In [204]:
percents_2p1 = num_grp_2.divide(den_grp_2,fill_value=0)


## MOST RECENT PERFORMANCE DATA BY PROGRAM

In [224]:
pf2p1a = zip(percents_2p1.index,percents_2p1.values)
for i in pf2p1a:
    print('{0:.4f}---{1:.2f}%'.format(i[0],i[1]*100))

11.0201---66.67%
11.0401---45.45%
11.0701---54.55%
11.1003---72.73%
15.0201---90.00%
15.0303---46.15%
15.0407---66.67%
15.0616---0.00%
15.0801---80.00%
15.0805---46.15%
15.1302---100.00%
31.0101---88.89%
43.0103---56.60%
43.0107---80.00%
43.0114---100.00%
43.0204---41.67%
44.0000---66.67%
45.0702---100.00%
47.0101---78.26%
47.0201---70.00%
48.0508---90.20%
48.0510---90.91%
50.0102---68.42%
50.0406---76.92%
50.0409---53.57%
50.0410---61.29%
51.0000---10.34%
51.0707---50.00%
51.0907---100.00%
51.0908---80.00%
51.1501---75.00%
51.3801---87.27%
52.0201---73.15%
52.0205---86.67%
52.0302---62.86%
52.0401---76.92%
52.0703---0.00%
52.0801---90.91%


## PROGRAMS NOT MEETING TARGET

In [247]:
p_unmet_2p1 = pd.DataFrame(percents_2p1[(percents_2p1.values<=target_2)]).reset_index().set_index('CTEA 1A / CIP Code / DYN||8774_DYN')
p_unmet_2p1_pf = percents_2p1[(percents_2p1.values<=target_2)]

In [249]:
pf2p1b = zip(p_unmet_2p1_pf.index,p_unmet_2p1_pf.values)
for i in pf2p1b:
    print('{0:.4f}---{1:.2f}%'.format(i[0],i[1]*100))

15.0616---0.00%
51.0000---10.34%
52.0703---0.00%


## SPECIAL POPULATIONS NOT MEETING TARGET WITHIN PROGRAMS

In [200]:
# Special Pops falling below 1P1 target, by CIP
unmet_spop_2p1 = spop_2p1.set_index('CTEA 1A / CIP Code / DYN||8774_DYN')

In [201]:
for items in rows_2p1:
    try:
        print(items.columns[1].split(' / ')[1])
        for v in items.values:
            cip = v[0]
            meas = v[1]
            print('{0:.4f}---{1:.2f}%'.format(cip,meas*100))

    except:
        print('NA')
    

Disabled
51.0000---20.00%
DISPLACED_HM
Economic Disadvantage
51.0000---8.00%
Homeless
Limited English
Migrant
Single Parent
43.0103---20.00%
Youth Aged Out
Youth Armed Forces


# 3P1 Performance Indicator

In [45]:
num_3p1 = completer_set[((completer_set['Female']=='Y') & (completer_set[gender_field]==2))|((completer_set['Male']=='Y') & (completer_set[gender_field]==1))]
num_grp_3 = num_3p1.groupby(cip_field)[id_field].count()
den_3p1 = completer_set[(completer_set['Female']=='Y') | (completer_set['Male']=='Y')]
den_grp_3 = den_3p1.groupby(cip_field)[id_field].count()
percents_3p1 = num_grp_3.divide(den_grp_3,fill_value=0)

In [46]:
# Calculate performance for each special population by CIP, return values below target
rows_3p1 = []
for i in spop_fields:

    lim_num = pd.DataFrame(num_3p1[(num_3p1[i]==1)])
    lim_den = pd.DataFrame(den_3p1[(den_3p1[i]==1)])

    num = pd.DataFrame(lim_num).filter([cip_field,i]).groupby(cip_field).count()
    dem = pd.DataFrame(lim_den).filter([cip_field,i]).groupby(cip_field).count()
    pct = num.divide(dem)
    underperf = pct[(pct.values<=target_3)]
    
    rows_3p1.append(underperf.reset_index())
    
spop_3p1 = pd.concat(rows_3p1).dropna(axis=1,how='all').groupby('CTEA 1A / CIP Code / DYN||8774_DYN').max().reset_index()

## MOST RECENT PERFORMANCE DATA BY PROGRAM

In [254]:
pf3p1a = zip(percents_3p1.index,percents_3p1.values)
for i in pf3p1a:
    print('{0:.4f}---{1:.2f}%'.format(i[0],i[1]*100))

11.0201---13.16%
11.0401---28.57%
11.0701---43.48%
11.1003---22.64%
13.1314---0.00%
15.0201---3.33%
15.0303---14.81%
15.0407---0.00%
15.0616---0.00%
15.0801---0.00%
15.0805---5.71%
15.1302---10.00%
43.0103---42.98%
43.0107---28.57%
43.0114---50.00%
45.0702---0.00%
47.0101---6.67%
47.0201---2.38%
48.0508---21.59%
48.0510---18.52%
51.0707---2.22%
51.1501---36.73%
51.3801---16.38%
52.0201---49.36%
52.0205---10.87%
52.0302---34.38%


## PROGRAMS NOT MEETING TARGET

In [255]:
p_unmet_3p1 = pd.DataFrame(percents_3p1[(percents_3p1.values<=target_3)]).reset_index().set_index('CTEA 1A / CIP Code / DYN||8774_DYN')
p_unmet_3p1_pf = percents_3p1[(percents_3p1.values<=target_3)]

In [257]:
pf3p1b = zip(p_unmet_3p1_pf.index,p_unmet_3p1_pf.values)
for i in pf3p1b:
    print('{0:.4f}---{1:.2f}%'.format(i[0],i[1]*100))

11.0201---13.16%
13.1314---0.00%
15.0201---3.33%
15.0303---14.81%
15.0407---0.00%
15.0616---0.00%
15.0801---0.00%
15.0805---5.71%
15.1302---10.00%
45.0702---0.00%
47.0101---6.67%
47.0201---2.38%
51.0707---2.22%
51.3801---16.38%
52.0205---10.87%


## SPECIAL POPULATIONS NOT MEETING TARGET WITHIN PROGRAMS

In [202]:
# Special Pops falling below 1P1 target, by CIP
unmet_spop_3p1 = spop_3p1.set_index('CTEA 1A / CIP Code / DYN||8774_DYN')


In [203]:
for items in rows_3p1:
    try:
        print(items.columns[1].split(' / ')[1])
        for v in items.values:
            cip = v[0]
            meas = v[1]
            print('{0:.4f}---{1:.2f}%'.format(cip,meas*100))

    except:
        print('NA')
    

Disabled
48.0508---10.00%
DISPLACED_HM
Economic Disadvantage
11.0201---14.29%
15.0201---4.00%
15.0303---12.50%
47.0201---3.12%
51.3801---15.91%
52.0205---15.00%
Homeless
Limited English
Migrant
Single Parent
51.3801---10.53%
52.0302---11.11%
Youth Aged Out
Youth Armed Forces


## RESULTS

In [50]:
# load measures into dataframe
df1a = pd.DataFrame(percents_1p1).reset_index()
df1a['Target 1'] = target_1
df1a.rename(columns={'CTEA 1A / Student ID / DYN||8761_DYN':'Measure 1P1'},inplace=True)
df1b = df1a.merge(p_unmet_1p1,on='CTEA 1A / CIP Code / DYN||8774_DYN',how='left')
df1b.rename(columns={'CTEA 1A / Student ID / DYN||8761_DYN':'Unmet 1P1'},inplace=True)
df1c =spop_1p1
df1c.rename(columns={'CTEA 1A / Disabled / DYN||8766_DYN':'Unmet 1P1 Disabled',
                     'CTEA 1A / DISPLACED_HM / DYN||8776_DYN': 'Unmet 1P1 Displaced',
                     'CTEA 1A / Economic Disadvantage / DYN||8777_DYN': 'Unmet 1P1 Econ Dis',
                     'CTEA 1A / Homeless / DYN||8769_DYN': 'Unmet 1P1 Homeless',
                     'CTEA 1A / Limited English / DYN||8780_DYN': 'Unmet 1P1 Lim Eng',
                     'CTEA 1A / Migrant / DYN||8772_DYN': 'Unmet 1P1 Migrant',
                     'CTEA 1A / Single Parent / DYN||8775_DYN': 'Unmet 1P1 Sing Par',
                     'CTEA 1A / Youth Aged Out / DYN||8779_DYN': 'Unmet 1P1 Aged Out',
                     'CTEA 1A / Youth Armed Forces / DYN||8762_DYN': 'Unmet 1P1 Armed Forces'
                     
                    },inplace=True)
df1 = df1b.merge(df1c,on='CTEA 1A / CIP Code / DYN||8774_DYN',how='left')



df2a = pd.DataFrame(percents_2p1).reset_index()
df2a['Target 2'] = target_2
df2a.rename(columns={'CTEA 1A / Student ID / DYN||8761_DYN':'Measure 2P1'},inplace=True)
df2b = df2a.merge(p_unmet_2p1,on='CTEA 1A / CIP Code / DYN||8774_DYN',how='left')
df2b.rename(columns={'CTEA 1A / Student ID / DYN||8761_DYN':'Unmet 2P1'},inplace=True)
df2c =spop_2p1
df2c.rename(columns={'CTEA 1A / Disabled / DYN||8766_DYN':'Unmet 2P1 Disabled',
                     'CTEA 1A / DISPLACED_HM / DYN||8776_DYN': 'Unmet 2P1 Displaced',
                     'CTEA 1A / Economic Disadvantage / DYN||8777_DYN': 'Unmet 2P1 Econ Dis',
                     'CTEA 1A / Homeless / DYN||8769_DYN': 'Unmet 2P1 Homeless',
                     'CTEA 1A / Limited English / DYN||8780_DYN': 'Unmet 2P1 Lim Eng',
                     'CTEA 1A / Migrant / DYN||8772_DYN': 'Unmet 2P1 Migrant',
                     'CTEA 1A / Single Parent / DYN||8775_DYN': 'Unmet 2P1 Sing Par',
                     'CTEA 1A / Youth Aged Out / DYN||8779_DYN': 'Unmet 2P1 Aged Out',
                     'CTEA 1A / Youth Armed Forces / DYN||8762_DYN': 'Unmet 2P1 Armed Forces'
                     
                    },inplace=True)
df2 = df2b.merge(df2c,on='CTEA 1A / CIP Code / DYN||8774_DYN',how='left')


df3a = pd.DataFrame(percents_3p1).reset_index()
df3a['Target 3'] = target_3
df3a.rename(columns={'CTEA 1A / Student ID / DYN||8761_DYN':'Measure 3P1'},inplace=True)
df3b = df3a.merge(p_unmet_3p1,on='CTEA 1A / CIP Code / DYN||8774_DYN',how='left')
df3b.rename(columns={'CTEA 1A / Student ID / DYN||8761_DYN':'Unmet 3P1'},inplace=True)
df3c =spop_3p1
df3c.rename(columns={'CTEA 1A / Disabled / DYN||8766_DYN':'Unmet 3P1 Disabled',
                     'CTEA 1A / DISPLACED_HM / DYN||8776_DYN': 'Unmet 3P1 Displaced',
                     'CTEA 1A / Economic Disadvantage / DYN||8777_DYN': 'Unmet 3P1 Econ Dis',
                     'CTEA 1A / Homeless / DYN||8769_DYN': 'Unmet 3P1 Homeless',
                     'CTEA 1A / Limited English / DYN||8780_DYN': 'Unmet 3P1 Lim Eng',
                     'CTEA 1A / Migrant / DYN||8772_DYN': 'Unmet 3P1 Migrant',
                     'CTEA 1A / Single Parent / DYN||8775_DYN': 'Unmet 3P1 Sing Par',
                     'CTEA 1A / Youth Aged Out / DYN||8779_DYN': 'Unmet 3P1 Aged Out',
                     'CTEA 1A / Youth Armed Forces / DYN||8762_DYN': 'Unmet 3P1 Armed Forces'
                     
                    },inplace=True)
df3 = df3b.merge(df3c,on='CTEA 1A / CIP Code / DYN||8774_DYN',how='left')

m1 = df1.merge(df2,on='CTEA 1A / CIP Code / DYN||8774_DYN',how='left')
measures = m1.merge(df3,on='CTEA 1A / CIP Code / DYN||8774_DYN',how='left')
measures.rename(columns={'CTEA 1A / CIP Code / DYN||8774_DYN':'CIP Code'},inplace=True)
measures['CIP Code'] = measures['CIP Code'].map('{:.4f}'.format)


In [52]:
print('Overall Performance for 1P1 is {0:.2f}%'.format(len(num_1p1)/len(den_1p1)*100))
print('Overall Performance for 2P1 is {0:.2f}%'.format(len(num_2p1)/len(den_2p1)*100))
print('Overall Performance for for 3P1 is {0:.2f}%'.format(len(num_3p1)/len(den_3p1)*100))

Overall Performance for 1P1 is 52.44%
Overall Performance for 2P1 is 69.35%
Overall Performance for for 3P1 is 27.19%


In [54]:
    
print(measures)
choice = input('Export measures to Excel? Y/N:')
if ((choice =='Y') | (choice =='y')):
    measures.to_excel('measures.xlsx',index=None)
    print('Exported to measures.xlsx')
else:
    
    pass

   CIP Code  Measure 1P1  Target 1  Unmet 1P1  Unmet 1P1 Disabled  \
0   11.0201     0.333333    0.4975   0.333333                 NaN   
1   11.0401     0.800000    0.4975        NaN                 NaN   
2   11.0701     0.666667    0.4975        NaN                 NaN   
3   11.1003     0.750000    0.4975        NaN                 NaN   
4   15.0201     0.888889    0.4975        NaN                 NaN   
5   15.0303     0.666667    0.4975        NaN                 NaN   
6   15.0407     0.000000    0.4975   0.000000                 NaN   
7   15.0801     1.000000    0.4975        NaN                 NaN   
8   15.0805     0.333333    0.4975   0.333333                 NaN   
9   15.1302     0.500000    0.4975        NaN                 NaN   
10  31.0101     0.625000    0.4975        NaN                 NaN   
11  43.0103     0.366667    0.4975   0.366667                 NaN   
12  43.0107     0.312500    0.4975   0.312500                 NaN   
13  43.0114     0.500000    0.4975

Export measures to Excel? Y/N:y
Exported to measures.xlsx


In [ ]:
measures